In [ ]:
# все импорты разом
import pandas as pd
import re
import numpy as np
from sklearn.metrics import homogeneity_completeness_v_measure, adjusted_rand_score, rand_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from scipy.optimize import linear_sum_assignment
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

# Кластеризация по place_labels: k-means + DBSCAN

In [ ]:
stem_clean = pd.read_excel("stem_clean.xlsx")

In [1]:
# фильтрую по id тексты с сомнительной атрибуцией к Улан-Батору
ids_to_remove = [5, 6, 8, 15, 35, 36, 42, 46, 49, 77, 80, 94, 95, 97, 99, 100]

stem_clean = stem_clean[~stem_clean['id'].isin(ids_to_remove)]

NameError: name 'stem_clean' is not defined

In [ ]:
# tf-idf
vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=2,
)
X_tfidf = vectorizer.fit_transform(stem_clean["text"])

print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

n_samples: 99, n_features: 635


## k-means

In [ ]:
kmeans = KMeans(
    n_clusters=7,
    max_iter=100,
    n_init=20,
    random_state=42
)

kmeans.fit(X_tfidf)
clusters = kmeans.fit_predict(X_tfidf)
stem_clean['cluster'] = clusters

In [ ]:
# топ-20 ближайших к центру слов для каждого кластера
terms = vectorizer.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

for i in range(kmeans.n_clusters):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i, :20]:
        print(terms[ind], end=" ")
    print()

Cluster 0: жить дом переезжать отец двор закрывать видеть знать китаец стоять 
Cluster 1: змея лошадь подъезжать пойти знать мож откуда вода поросенок сидеть 
Cluster 2: рассказывать сделать бабка умирать ребенок помнить ведьма находить лечить поросенок 
Cluster 3: машина выходить ящик ребенок дверь длинный свечка просто знать ехать 
Cluster 4: юрта деньги отдавать баба икона мама мимо сторона описывать бабушка 
Cluster 5: заговаривать грыжа сучок бабка полено ходить испуг тетя вообще выбрасывать 
Cluster 6: петух корова червь палка доить курочка вылупливаться приносить земля пойти 


In [ ]:
# преобразование ожидаемых меток для вычисления метрик соответствия ожиданиям
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(stem_clean['place_labels'])

In [ ]:
# метрики: гомогенность, полнота, v-мера, индекс adjusted rand
print(homogeneity_completeness_v_measure(y, kmeans.labels_))
print(adjusted_rand_score(y, kmeans.labels_))

(np.float64(0.20047713897259842), np.float64(0.1741489039446749), np.float64(0.18638786426149184))
0.014183651584323243


In [ ]:
# распределение текстов по кластерам
labels = kmeans.labels_
unique, counts = np.unique(labels, return_counts=True)

for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} текстов")


clusters_ids = stem_clean.groupby("cluster")["id"].apply(list)
for cluster_num, ids in clusters_ids.items():
    print(f"Кластер {cluster_num}: {', '.join(map(str, ids))}")

Кластер 0: 1, 2, 3, 4, 9, 10, 14, 20, 21, 22, 78, 79, 81, 82, 86, 87, 89, 91, 96, 102, 107, 110, 113
Кластер 1: 7, 60, 61, 62, 63, 64, 65, 69, 70, 109, 111
Кластер 2: 13, 23, 24, 25, 28, 30, 37, 41, 43, 47, 48, 52, 58, 59, 68, 72, 74, 75, 76, 88, 92, 98, 103
Кластер 3: 12, 16, 17, 18, 19, 26, 27, 38, 50, 51, 53, 55, 56, 73, 84, 85, 101, 112, 114
Кластер 4: 11, 29, 54, 71, 83, 90, 93, 104, 105, 106, 108, 115
Кластер 5: 31, 32, 33, 34, 45, 57
Кластер 6: 39, 40, 44, 66, 67


## DBSCAN

In [ ]:
# снижаю размерность
np.random.seed(42)
pca = PCA(n_components=2)
pca_data = pca.fit_transform(X_tfidf)

Кластер -1: 3 текстов
Кластер 0: 83 текстов
Кластер 1: 3 текстов
Кластер 2: 2 текстов
Кластер 3: 2 текстов
Кластер 4: 2 текстов
Кластер 5: 2 текстов
Кластер 6: 2 текстов


In [ ]:
# кластеризация DBSCAN
db = DBSCAN(eps=0.09, min_samples=2)
db.fit(pca_data)

In [ ]:
# метрики: гомогенность, полнота, v-мера, индекс adjusted rand
print(homogeneity_completeness_v_measure(y, db.labels_))
print(adjusted_rand_score(y, db.labels_))

(np.float64(0.10040510107404757), np.float64(0.210067332434913), np.float64(0.13586927191636972))
0.014183651584323243


In [ ]:
# распределение текстов по кластерам
labels = db.labels_
stem_clean['cluster'] = labels
unique, counts = np.unique(labels, return_counts=True)

for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} текстов")

clusters_ids = stem_clean.groupby("cluster")["id"].apply(list)

for cluster_num, ids in clusters_ids.items():
    print(f"Кластер {cluster_num}: {', '.join(map(str, ids))}")

Кластер -1: 45, 57, 65
Кластер 0: 1, 2, 3, 4, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 37, 38, 39, 40, 41, 43, 44, 47, 48, 50, 51, 52, 53, 54, 55, 56, 58, 59, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 96, 98, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 115
Кластер 1: 31, 32
Кластер 2: 33, 34
Кластер 3: 60, 62
Кластер 4: 61, 64
Кластер 5: 63, 111


# Кластеризация сводного корпуса

In [ ]:
experiment = pd.read_excel("experiment.xlsx")

In [ ]:
# tf-idf
vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
)
X_tfidf = vectorizer.fit_transform(experiment["text"])

In [ ]:
# кластеризация k-means
kmeans = KMeans(
    n_clusters=2,
    max_iter=100,
    n_init="auto",
    random_state=0
)

kmeans.fit(X_tfidf)
clusters = kmeans.fit_predict(X_tfidf)
experiment['cluster'] = clusters

In [ ]:
# распределение по кластерам
labels = kmeans.labels_
unique, counts = np.unique(labels, return_counts=True)

for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} текстов")

Кластер 0: 122 текстов
Кластер 1: 108 текстов


In [ ]:
# преобразование ожидаемых меток
experiment['label'] = experiment['id'].apply(lambda x: 2 if 'zin' in str(x) else 1)
y = experiment['label']

In [ ]:
# метрики
print(homogeneity_completeness_v_measure(y, kmeans.labels_))
print(rand_score(y, kmeans.labels_))

(np.float64(0.5083364920358026), np.float64(0.5096995920685227), np.float64(0.5090171294903545))
0.8053920637934308


In [ ]:
# матрица ошибок
cm = confusion_matrix(y, kmeans.labels_)

# наилучшее соответствие кластеров меткам
row_ind, col_ind = linear_sum_assignment(-cm)

# количество правильных
correct = cm[row_ind, col_ind].sum()
total = len(y)
incorrect = total - correct

print(f"Правильно кластеризовано: {correct} из {total}")
print(f"Ошибок: {incorrect}")
print(f"Точность: {correct / total:.3f}")

Правильно кластеризовано: 205 из 230
Ошибок: 25
Точность: 0.891


In [ ]:
# топ-20 ближайших к центру слов для каждого кластера
terms = vectorizer.get_feature_names_out()  # слова из векторизатора
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]  # индексы слов по убыванию важности

for i in range(kmeans.n_clusters):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i, :20]:
        print(terms[ind], end=" ")
    print()

Cluster 0: пойти идти конь приходить баня становиться гыт взять черт давать 
Cluster 1: знать змея бабка ребенок заговаривать дом просто рассказывать жить помнить 
